## 1. 今日の量子コンピュータの問題

- Noisy Intermediate-Scale Quantum (NISQ) デバイス
    - 量子回路が深くなる（ゲート数が多くなる）ほど、誤差が大きくなる
    - 十分な量子ビット数ではない
- 量子コンピュータは特別な量子ゲートしか用意されていない。量子デバイスごとに異なる  
  Quantinuum H-series: default: {Rz, PhasedX, ZZMax, ZZPhase}, option: {Rz, PhasedX, SU(4)},  
  IBM Quantum: Heron {CZ, ID, RX, RZ, RZZ, SX, X}, Eagle {ECR, ID, RZ, SX, X}  
- 量子コンピュータによっては特定の２量子ビット間のエンタングルゲートしか用意されていない  
  Quantinuumの量子コンピュータは全結合なので、任意の二つの量子ビットをエンタングルさせることが可能
- それぞれの量子コンピュータに対して、量子ソフトウェアツールキットが用意されてる


### 1-1. TKETとは
- Quantum Software Development Kit
- TKETに実装されている量子回路最適化はC++で実装。現在開発中のTKET2ではRustで実装。
- pythonモジュール　ユーザーはPythonパッケージ`pytket`で量子プログラミングの開発が可能
- 最適化コンパイラ：　ユーザーフレンドリーな回路→量子デバイスで実行可能な回路に変換可能
    - Language-agnostic (多くの量子プログラミングフレームワーク(qiskit, Cirq, etc)をサポート)
    - Retagetable (多くの量子デバイス(Quantinuum, IBM, Amazon Braket(IonQ, Rigetti, IQM), Microsoft Azure Quantum(Quantinuu,IonQ, Rigetti) etc)をサポート)
    - Circuit Optimisation (量子計算時に生じるデバイスエラーの影響を最小化。デバイス依存＆デバイス非依存のものが実装)
    
<img src="./fig/tket1.png" width="750">



#### 参照
- [pytket ドキュメント](https://docs.quantinuum.com/tket/)
- [pytket ユーザーガイド](https://docs.quantinuum.com/tket/user-guide/)
- [t|ket⟩ : A Retargetable Compiler for NISQ Devices](https://arxiv.org/abs/2003.10611)
- [TKET slack channel](https://join.slack.com/t/tketusers/shared_invite/zt-2aoan2s87-WDdZQeY2dbJQgAQE6O~3qg)

<img src="./fig/slack-qr.png" width="250">


### 1-2. pytketと拡張 pytket (python パッケージ)
Python 3.10.11で動作確認をしています。

|  パッケージ |  概要  |
| :---- | :---- |
|  pytket  |  TKETを利用するためのpython モジュール  ( available for python3.10 or higher )|
|  pytket-quantinuum  |  Quantinuumデバイス、エミュレータを利用するためのpytket-extension  |
|  pytket-qiskit  |  qiskit、IMBQデバイスを利用するためのpytket-extension  |
|  pytket-azure  |  Azure Quantumを利用するためのpytket-extension  |
|  pytket-braket  |  Amazon Braketを利用するためのpytket-extension  |
|  pytket-circ    |  Google circを利用するためのpytket-extension  |
|  pytket-qulacs  |  Qulacsシミュレータを利用するためのpytket-extension  |

<img src="./fig/tket2.png" width="850">

## 2. 量子回路を作成して、シュミレータや量子コンピュータにジョブを実行　（より詳しい内容は２日目に行います）
ここでは、量子デバイスやシュミレーションを利用するまでの一連の流れをと`TKET`で作成したBell状態の量子回路で見てみる。

pytket, pytket-extensionのインストール

In [ ]:
#!pip install -U pytket pytket-quantinuum pytket-quantinuum[pecos] 

In [ ]:
!pip freeze |grep pytket

### 2-1 `TKET`でBell状態を作成
$$ |\Psi\rangle = \frac{1}{\sqrt{2}}(|00\rangle+|11\rangle)$$

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

bell = Circuit(2)
bell.H(0).CX(0,1)
bell.measure_all()
render_circuit_jupyter(bell)

Note：TKETの可視化では、可視化した量子回路の画像ファイルを出力できる。

In [ ]:
#bell.to_latex_file('bell.tex')

### 2-2. `pytket-quantinuum`でTKET 量子回路をQuantinuum エミュレータで計算

In [ ]:
#!pip install -U pytket-quantinuum

#### ロカールマシンでQuantinuum エミュレータを利用するための設定

In [ ]:
from pytket.extensions.quantinuum import QuantinuumAPIOffline
api = QuantinuumAPIOffline()

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend
backendlist= [b.device_name for b in QuantinuumBackend.available_devices(api_handler = api)]
backendlist

#### a. エミュレータまたはQuantinuum 量子コンピュータを選択  
Quantinuum 量子コンピュータへのアクセスはマシンタイムの購入が必要

In [ ]:
quantinuum_backend = QuantinuumBackend(device_name ='H1-1LE',api_handler = api)

In [ ]:
quantinuum_backend.backend_info

#### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
quantinuum_bell = quantinuum_backend.get_compiled_circuit(bell)
render_circuit_jupyter(quantinuum_bell)
render_circuit_jupyter(bell)

#### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
handle = quantinuum_backend.process_circuit(quantinuum_bell, n_shots=1000)
result = quantinuum_backend.get_result(handle)

In [ ]:
from qiskit.visualization import plot_histogram
counts = result.get_counts()
plot_histogram(counts)

### 2-3. `pytket-qiskit`でTKET 量子回路をIBM ローカルシミュレータ,IBM 量子コンピュータで計算

In [ ]:
#!pip install -U pytket-qiskit

#### IBM ローカルシミュレータにジョブを実行

##### a. エミュレータまたはQuantinuum 量子コンピュータを選択

In [ ]:
from pytket.extensions.qiskit import AerBackend
backend = AerBackend()

##### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
ibm_bell = backend.get_compiled_circuit(bell)

##### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
handle = backend.process_circuit(ibm_bell, n_shots =1000)
result_counts = backend.get_result(handle).get_counts()
plot_histogram(result_counts)

#### IBMの量子デバイスにジョブを実行

IBM Quantum device の情報, IBM Quantum トークンの取得  
https://quantum.ibm.com/

##### IBM Quantum トークンの設定

In [ ]:
path = 'key/ibm-token'
f = open(path)
ibm_token = f.read()
f.close()
from pytket.extensions.qiskit.backends.config import set_ibmq_config
set_ibmq_config(ibmq_api_token=ibm_token, instance=f"ibm-q/open/main")

In [ ]:
from pytket.extensions.qiskit import IBMQBackend

In [ ]:
backendinfo_list = IBMQBackend.available_devices(instance=f"ibm-q/open/main")

In [ ]:
[dev.device_name for dev in backendinfo_list]

#### a. エミュレータまたはQuantinuum 量子コンピュータを選択

In [ ]:
ibm_backend = IBMQBackend("ibm_brisbane")
ibm_backend.backend_info.device_name

#### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
ibm_bell = ibm_backend.get_compiled_circuit(bell)
render_circuit_jupyter(ibm_bell)
render_circuit_jupyter(bell)

#### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
handle = ibm_backend.process_circuit(ibm_bell, n_shots =1000)
#result = ibm_backend.get_result(handle)
#counts = result.get_counts()
#plot_histogram(counts)

In [ ]:
#from pytket.backends import ResultHandle
#from pytket.extensions.qiskit import IBMQBackend
#ibm_backend = IBMQBackend("ibm_kyiv")

In [ ]:
#handle_id = ('cx8kz9xbqkhg0088wra0', 0, 2, 'null')
#handle = ResultHandle.from_str(str(handle_id))
#result = ibm_backend.get_result(handle)
#counts = result.get_counts()
#print(counts)
#from qiskit.visualization import plot_histogram
#plot_histogram(counts)

### 2-4. `pytket-qulacs`でTKET 量子回路をQulacsシミュレータで計算

In [ ]:
#!pip install -U pytket-qulacs

#### a. エミュレータまたはQuantinuum 量子コンピュータを選択

In [ ]:
from pytket.extensions.qulacs import QulacsBackend
qulacs_backend = QulacsBackend()

In [ ]:
qulacs_backend.backend_info

#### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
qulacs_bell = qulacs_backend.get_compiled_circuit(bell)

#### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
handle = qulacs_backend.process_circuit(bell, n_shots =1000)
result_counts = qulacs_backend.get_result(handle).get_counts()
plot_histogram(result_counts)

#### GPU上でQulacsを利用している場合にも対応している


In [ ]:
#from pytket.extensions.qulacs import QulacsGPUBackend
#qualcs_backend = QulacsGPUBackend()
#qulacs_bell = qulacs_backend.get_compiled_circuit(bell)
#handle = qulacs_backend.process_circuit(qulacs_bell, n_shots =1000)
#result_counts = qulacs_backend.get_result(handle).get_counts()
#plot_histogram(result_counts)

詳しくは
https://tket.quantinuum.com/extensions/pytket-qulacs/
を参照ください

### 2-5. `pytket-azure`でTKET 量子回路をMicrosoft Azure Quantum(有料)にある量子デバイスで計算
本講演では詳細は割愛致しますが、Azure Quantum上でTKETをご利用になられたい方は下記のページを参考に設定をしてください。  
https://tket.quantinuum.com/extensions/pytket-azure/
または  
米澤(yasuyoshi.yonezawa@quantinuum.com)までご連絡ください。

In [ ]:
#!pip install pytket-azure

#### Azure Quantum connection_stringの設定
connection_string は下記を参照して取得してください。  
https://learn.microsoft.com/en-us/azure/quantum/how-to-connect-workspace

In [ ]:
path = 'key/azure-key'
f = open(path)
connection_string = f.read()
f.close()
# Azure Qauntum上の量子デバイスまたはシミュレータを利用する
from pytket.extensions.azure import AzureBackend, AzureConfig, set_azure_config
set_azure_config(connection_string= connection_string, use_string = True)

In [ ]:
backendinfo_list = [device.device_name for device in AzureBackend.available_devices()]
backendinfo_list

#### a. エミュレータまたはQuantinuum 量子コンピュータを選択

In [ ]:
# Quantinuum H1 device
# azure_backend = AzureBackend(name='quantinuum.qpu.h1-1')
# Quantinuum H1 emulator
# azure_backend = AzureBackend(name='quantinuum.sim.h1-1e')
# Quantinuum H1 Syntax Checker
azure_backend = AzureBackend(name='quantinuum.sim.h1-1sc')

# Rigetti device
# azure_backend = AzureBackend(name='rigetti.qpu.ankaa-9q-3')
# Rigetti simulator
# azure_backend = AzureBackend(name='rigetti.sim.qvm')

# IonQ device
# azure_backend = AzureBackend(name='ionq.qpu.aria-1')
# azure_backend = AzureBackend(name='ionq.qpu.aria-2')
# IonQ simulator
# azure_backend = AzureBackend(name='ionq.simulator')

In [ ]:
azure_backend.backend_info

#### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
azure_bell = azure_backend.get_compiled_circuit(bell)
render_circuit_jupyter(azure_bell)

#### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
result = azure_backend.run_circuit(azure_bell, n_shots=100)
counts = result.get_counts()
counts

### 2-6. `pytket-braket`でTKET 量子回路をBraketシミュレータやAmazon Braket(有料)にある量子デバイスで計算
本講演では詳細は割愛致しますが、Amazon Bracket上でTKETをご利用になられたい方は下記のページを参考に設定をしてください。  
https://tket.quantinuum.com/extensions/pytket-braket/  
または  
米澤(yasuyoshi.yonezawa@quantinuum.com)までご連絡ください。

#### A.ローカル環境からBraketにアクセスされている方は、keyフォルダーのaws-keyのアクセスキー等の情報を入力し下記を実行してください

In [ ]:
#!pip install -U pytket-braket

In [ ]:
path = 'key/aws-key'
f = open(path)
aws_access_key_id, aws_secret_access_key, s3_name, bucket_key= [s.strip() for s in f.readlines()]
f.close()

In [ ]:
import boto3
from braket.aws import AwsSession
boto_session = boto3.Session(
    aws_access_key_id= aws_access_key_id,
    aws_secret_access_key= aws_secret_access_key,
    region_name= 'us-east-1'
#    region_name= 'us-west-1'
#    region_name= 'eu-north-1'
)
## us-east-1: IonQ, Simulators
## us-west-1: Rigetti, Simulators
## eu-north-1: IQM, Simulators
##
aws_session = AwsSession(boto_session=boto_session)

In [ ]:
[dev for dev in aws_session.search_devices() if dev["deviceStatus"]!='RETIRED']

#### B. Sagemaker StudioからBraketにアクセスされる方は下記を実行してください。

In [ ]:
#!pip install pytket-braket

In [ ]:
#import boto3
#from braket.aws import AwsSession
#boto_session = boto3.Session(
#    aws_access_key_id= aws_access_key_id,
#    aws_secret_access_key= aws_secret_access_key,
#    region_name= 'us-east-1'
#    region_name= 'us-west-1'
#    region_name= 'eu-north-1'
#)
# us-east-1: IonQ, Simulators
# us-west-1: Rigetti, Simulators
# eu-west-2: IQM
#
#aws_session = AwsSession(boto_session=boto_session)

In [ ]:
#[dev for dev in aws_session.search_devices() if dev["deviceStatus"]!='RETIRED']

#### a. エミュレータまたはQuantinuum 量子コンピュータを選択

In [ ]:
# backend の選択
from pytket.extensions.braket import BraketBackend

# ionQ   arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1, region: us-east-1
# aws_backend = BraketBackend(device='Aria-1',region = 'us-east-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)
# ionQ   arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2, region: us-east-1
# aws_backend = BraketBackend(device='Aria-2',region = 'us-east-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)
# ionQ   arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1, region: us-east-1
# aws_backend = BraketBackend(device='Forte-1',region = 'us-east-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)

# Ankaa-2   arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-2 #Ankaa-2, region: us-west-1
# aws_backend = BraketBackend(device='Ankaa-2',region = 'us-west-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'rigetti', aws_session=aws_session)

# IQM Garnet arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet
# aws_backend = BraketBackend(device='Garnet',region = 'eu-north-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'iqm', aws_session=aws_session)

# SV1    sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'     #SV1, region:
# aws_backend = BraketBackend(device='sv1', s3_bucket=s3_name , s3_folder = bucket_key, aws_session=aws_session)
# TN1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/tn1'    #TN1, region: us-west-2, us-east-1
# aws_backend = BraketBackend(device='tn1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# DM1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/dm1'    #DM1, region: 
# aws_backend = BraketBackend(device='dm1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# Local sv1 simulator
aws_backend = BraketBackend(local = True)

In [ ]:
aws_backend.backend_info

#### b. 量子コンピュータ、エミュレータが用意しているゲートセットで量子回路を書き替える

In [ ]:
aws_bell = aws_backend.get_compiled_circuit(bell)
render_circuit_jupyter(aws_bell)

#### c. 量子コンピュータ、エミュレータにジョブを実行し、計算結果を取得

In [ ]:
aws_handle = aws_backend.process_circuit(aws_bell, n_shots =100)
aws_result = aws_backend.get_result(aws_handle)
plot_histogram(aws_result.get_counts())

## 3. 量子回路の最適化
量子計算時に生じるデバイスエラーの影響を最小化。  
デバイス非依存の最適化とデバイス依存の最適化（実はすでに上記で利用）がある。  
詳しくは２日目にご紹介します。

### 3-1. `PauliSquash` 関数を利用した、量子回路の最適化
TKETには量子回路を最適化する様々な機能が用意されている。
ここで`PauliSquash` 関数を利用した回路の最適化（デバイス非依存）を行う。
`PauliSquash` 関数：Pauli ゲートとCliffordゲートで表現された量子回路を出力）

ランダムな量子回路を作成し、回路の深さとCXの深さを数える。

In [ ]:
from pytket.pauli import Pauli
from pytket.circuit import PauliExpBox, fresh_symbol, OpType
from pytket.passes import DecomposeBoxes
box = PauliExpBox([Pauli.I, Pauli.Z, Pauli.X, Pauli.Y], fresh_symbol('tm'))
from pytket.utils import Graph
import numpy as np

def get_random_pauli_gadgets(n_qubits, n_pauli_gadgets, max_entangle):
    """ """
    paulis = [Pauli.I, Pauli.X, Pauli.Y, Pauli.Z]
    circ = Circuit(n_qubits)
    for i in range(n_pauli_gadgets):
        ls_paulis = [np.random.choice(paulis) for k in range(max_entangle)]
        if ls_paulis.count(Pauli.Y) % 2 == 0:
            continue
        if len(ls_paulis) - ls_paulis.count(Pauli.I) <= 1:
            continue
        qubits = np.random.choice(
            [i for i in range(n_qubits)], size=max_entangle, replace=False
        )
        box = PauliExpBox(ls_paulis, fresh_symbol('a'))
        circ.add_pauliexpbox(box, sorted(qubits))
    DecomposeBoxes().apply(circ)
    return circ

ランダムな量子ゲート（Pauliガジェット）を含んだ量子回路を作成

In [ ]:
circ = get_random_pauli_gadgets(
    n_qubits=8, n_pauli_gadgets=200, max_entangle=4
)
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)

`PauliSquash` 関数を使って、量子回路の最適化

In [ ]:
# Circuit optimization by using compiler passes.
from pytket.passes import PauliSquash
circx = circ.copy()
PauliSquash().apply(circx)
#FullPeepholeOptimise().apply(circx)
print('Circuit depth: ', circx.depth())
print('CX depth: ', circx.depth_by_type(OpType.CX))
render_circuit_jupyter(circx)

## 4. 量子回路の変換
pytketでは
- qiskitで記述した量子回路(`qiskit.QuantumCircuit`)からTKETの量子回路のクラスに変換が可能
- TKETで記述した量子回路からqiskitの量子回路(`qiskit.QuantumCircuit`)のクラスに変換が可能
- TKETで記述した量子回路と他の量子プログラミング言語(一部)での量子回路の交換が可能

参照：[pytket-extensions](https://tket.quantinuum.com/api-docs/extensions.html) 

### 4-1. `qiskit`の量子回路から`TKET`の量子回路に変換

In [ ]:
from qiskit import QuantumCircuit

In [ ]:
q_bell = QuantumCircuit(2)
q_bell.h(0)
q_bell.cx(0,1)
q_bell.measure_all()
q_bell.draw('mpl')

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk

In [ ]:
bell2 = qiskit_to_tk(q_bell)
bell2

In [ ]:
render_circuit_jupyter(bell2)

### 4-2. `TKET`の量子回路から`qiskit`の量子回路に変換

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit

In [ ]:
q_bell2 = tk_to_qiskit(bell)
q_bell2

In [ ]:
q_bell2.draw('mpl')

## 文献等

弊社Quantinuumのご紹介
- Quantinuum ウェブサイト（ 英語 ）： https://www.quantinuum.com/
- Quantinuum K.K. ウェブサイト（ 日本語 ）： https://quantinuum.co.jp/
- ニュース（ 日本語 ）： https://quantinuum.co.jp/news/  
- X（ 日本語 ）： https://x.com/quantinuum_jp?lang=en
- Quantinuum K.K.主催の勉強会（ 日本語 ）： https://quantinuum.connpass.com/  
- 採用情報：https://jobs.eu.lever.co/quantinuum?location=Japan%20Tokyo
- TKET slack channel：[TKET slack channel](https://join.slack.com/t/tketusers/shared_invite/zt-2aoan2s87-WDdZQeY2dbJQgAQE6O~3qg)

<img src="./fig/slack-qr.png" width="250">
